In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FuncFormatter

In [ ]:
# Connecting Google Drive
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Changing work folder
%cd /content/drive/MyDrive/Mate_homework

# **Data overview.**

In [ ]:
# Uploading dataset
df_countries = pd.read_csv("countries.csv")
df_countries.head()



In [ ]:
df_events = pd.read_csv("events.csv")
df_events.head()



In [ ]:
df_products = pd.read_csv("products.csv")
df_products.head()



### **Table: countries**

This table contains information about countries, including:

* **name** – the name of the country
* **alpha2**, **alpha3** – two- and three-letter country codes
* **region**, **sub-region** – the region and sub-region to which each country belongs

The **countries** table is joined with the **events** table using the fields **alpha3** (from *countries*) and **Country Code** (from *events*).

---

### **Table: events**

This table contains detailed information about sales events, including:

* **Order ID** – unique order identifier
* **Order Date** – date when the order was placed
* **Ship Date** – date when the order was shipped
* **Order Priority** – priority assigned to the order
* **Country Code** – three-letter country code
* **Product ID** – identifier of the product
* **Sales Channel** – sales channel used
* **Units Sold** – number of units sold
* **Unit Price** – price per unit
* **Unit Cost** – cost per unit

The **events** table is joined with the **countries** table using **Country Code** = **alpha3**, and with the **products** table using **Product ID** = **ID**.

---

### **Table: products**

This table contains product information:

* **id** – product identifier
* **item_type** – category or type of the product

It connects to the **events** table through **Product ID** (events) = **id** (products).


# **Data cleaning.**

In [ ]:
# Quantity of rows and columns in datasets
print("Table countries:", df_countries.shape)
print("Table events:", df_events.shape)
print("Table products:", df_products.shape)

In [ ]:
#Quantity of missing values
print("Table countries:", "\n", df_countries.isna().sum())
print("\n", "\n", "Table events:", "\n", df_events.isna().sum())
print("\n", "\n", "Table products:", "\n", df_products.isna().sum())


In [ ]:
#Persentage of missing values
print("Table countries:", "\n", df_countries.isna().sum() / df_countries.shape[0] * 100)
print("\n", "\n", "Table events:", "\n", df_events.isna().sum() / df_events.shape[0] * 100)
print("\n", "\n", "Table products:", "\n", df_products.isna().sum() / df_products.shape[0] * 100)

In [ ]:
# Analisys of missing values
df_countries[df_countries["alpha-2"].isna()]

In [ ]:
# Assigning the value of "alpha-2" for the country of Namibia
df_countries.loc[df_countries["alpha-3"] == "NAM", "alpha-2"] = "NA"


In [ ]:
df_countries[df_countries["alpha-2"].isna()]

In [ ]:
df_countries[df_countries["region"].isna()]

In [ ]:
df_countries[df_countries["sub-region"].isna()]

In [ ]:
#Delete rows that contain omitted value in countries dataframe
df_countries = df_countries.dropna()

# filling gaps in the events dataframe in the "Country Code" column with the "Unknown"
df_events.fillna({"Country Code": "Unknown"}, inplace=True)

#filling gaps in the events dataframe in the "Units Sold" column with the на median value
df_events.fillna({"Units Sold": df_events["Units Sold"].median()}, inplace=True)

In [ ]:
#Persentage of missing values in the dataframes after working with missing values
print("Table countries:", "\n", df_countries.isna().sum() / df_countries.shape[0] * 100)
print("\n", "\n", "Table events:", "\n", df_events.isna().sum() / df_events.shape[0] * 100)
print("\n", "\n", "Table products:", "\n", df_products.isna().sum() / df_products.shape[0] * 100)



* In the **countries** table, the value in the *alpha-2* column for **Namibia** was incorrectly interpreted by Pandas as **NaN**, even though the correct code should be **“NA”**. To fix this issue, the proper value was manually assigned.

* The missing values in the *region* and *sub-region* columns corresponded to **Antarctica**, which is not a relevant country for product sales. Therefore, these rows were removed from the dataset.

* In the **events** table, missing values in the *Country Code* column were replaced with **“Unknown”**, since the proportion of missing records was relatively large. Missing values in *Units Sold* were filled using the **median**, which is less sensitive to outliers and thus minimizes the impact on subsequent statistical calculations.


In [ ]:
#General information about datasets
df_countries.info()
df_events.info()
df_products.info()

In [ ]:
#Types of data in the datasets
print("Table countries:", "\n", df_countries.dtypes)
print("\n", "\n", "Table events:", "\n", df_events.dtypes)
print("\n", "\n", "Table products:", "\n", df_products.dtypes)

The **Order Date** and **Ship Date** columns have the incorrect data
type.

In [ ]:
#Converting the type of data for "Order Date" and "Ship Date" columns
df_events["Order Date"] = pd.to_datetime(df_events["Order Date"])
df_events["Ship Date"] = pd.to_datetime(df_events["Ship Date"])

In [ ]:
print(df_events.dtypes)

In [ ]:
#Checking for dublicates
duplicate_rows = df_countries.duplicated()
print(duplicate_rows)
print("Table countries:", duplicate_rows.sum())

duplicate_rows = df_events.duplicated()
print("\n","\n",duplicate_rows)
print("Table events:", duplicate_rows.sum())

duplicate_rows = df_products.duplicated()
print("\n","\n",duplicate_rows)
print("Table products:", duplicate_rows.sum())

The sum of logical values is 0, which means that all values are False. Therefore, **duplicate values are absent**.

In [ ]:
#Examining data for anomalies.
df_events.describe()


In [ ]:
#Searching for anomalies with Z-Score
z_score_units=(df_events["Units Sold"] - df_events["Units Sold"].median()) / df_events["Units Sold"].std()
q1=(z_score_units.abs() > 3).sum()
print("Quantity of anomal Units Sold values according to Z-Score:", q1)

z_score_price=(df_events["Unit Price"] - df_events["Unit Price"].median()) / df_events["Unit Price"].std()
q2=(z_score_price.abs() > 3).sum()
print("Quantity of anomal Unit Price values according to Z-Score:", q2)

z_score_cost=(df_events["Unit Cost"] - df_events["Unit Cost"].median()) / df_events["Unit Cost"].std()
q3=(z_score_cost.abs() > 3).sum()
print("Quantity of anomal Units Cost values according to Z-Score:", q3)


According to the results of descriptive statistics and additional calculations of the Z-score, **the anomalies are not revealed.**


# **Data analysis and visualization.**

In [ ]:
#Joining of three dataframes
df_events_products = pd.merge(df_events, df_products, left_on="Product ID", right_on="id", how="left")
df_events_products_countries = pd.merge(df_events_products, df_countries, left_on="Country Code", right_on="alpha-3", how="left")
df_events_products_countries.head()

In [ ]:
#To eliminate NaN values in the joining table, added the country "Unknown" in the "countries" table
df_countries.loc[len(df_countries.index)] = ["Unknown", "Unknown", "Unknown", "Unknown", "Unknown"]
df_countries.tail()

In [ ]:
#Rejoining of three tables after adding the Unknown country
df_events_products = pd.merge(df_events, df_products, left_on="Product ID", right_on="id", how="left")
df_events_products_countries = pd.merge(df_events_products, df_countries, left_on="Country Code", right_on="alpha-3", how="left")
df_events_products_countries.head()

In [ ]:
#Removing redundant columns
df_events_products_countries=df_events_products_countries.drop(["id", "alpha-3"], axis=1)
df_events_products_countries.head()

In [ ]:
#Rename of the table columns
df_events_products_countries.rename(
    columns={
        "name": "Country",
        "alpha-2": "Country Code 2",
        "item_type": "Category",
        "sub-region": "sub_region"
    },
    inplace=True
)


In [ ]:
df_events_products_countries.columns = (
    df_events_products_countries.columns
        .str.lower()
        .str.replace(" ", "_")
)


In [ ]:
df_events_products_countries.head()

In [ ]:
#Key company performance metrics

#Total number of orders
print("Total number of orders:", df_events_products_countries["order_id"].nunique())

#Total revenue
df_events_products_countries['total_revenue'] = df_events_products_countries['unit_price'] * df_events_products_countries['units_sold']
total_revenue = df_events_products_countries['total_revenue'].sum()
print("Total revenue:", total_revenue)

#Total costs
df_events_products_countries['total_cost'] = df_events_products_countries['unit_cost'] * df_events_products_countries['units_sold']
total_cost = df_events_products_countries['total_cost'].sum()
print("Total costs:", total_cost)

#Total profit
total_profit = total_revenue - total_cost
print("Total profit:", total_profit)

#Total countries covered
total_countries_covered = df_events_products_countries["country"].nunique()

# Exclude 'Unknown' if it was added as a placeholder for missing country codes
if 'unknown' in df_events_products_countries['country'].str.lower().unique():
    total_countries_covered -= 1
print("Total countries covered:", total_countries_covered)

In [ ]:
df_events_products_countries.info()

## **Key company performance metrics 2010 - 2017**

**Total number of order**s: 1330

**Total revenue**: 1704628370.65 USD

**Total costs**: 1202785737.53 USD

**Total profit**: 501842633.12 USD

**Total countries covered** (exclude "Uknown"): 45

## **Sales Analysis (revenue, costs, profits, product popularity) by Product Category.**


In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 10))

# Prepare columns for all graphs
df_events_products_countries["total_cost"] = (
    df_events_products_countries["unit_cost"] * df_events_products_countries["units_sold"]
)

df_events_products_countries["total_profit"] = (
    df_events_products_countries["total_revenue"] - df_events_products_countries["total_cost"]
)


#Total Revenue by Category
grouped_by_category = (
    df_events_products_countries.groupby("category")["total_revenue"]
    .sum().sort_values(ascending=True)
)

ax[0, 0].barh(grouped_by_category.index, grouped_by_category.values, edgecolor="black")
ax[0, 0].set_title("Total Revenue by Category")
ax[0, 0].set_xlabel("Total Revenue ($)")
ax[0, 0].set_ylabel("Category")

for i, value in enumerate(grouped_by_category.values):
    ax[0, 0].text(value * 1.01, i, f"{value:,.0f}", va="center")


#Total Profit by Category
grouped_profit_by_category = (
    df_events_products_countries.groupby("category")["total_profit"]
    .sum().sort_values(ascending=True)
)

ax[0, 1].barh(grouped_profit_by_category.index, grouped_profit_by_category.values, edgecolor="black")
ax[0, 1].set_title("Total Profit by Category")
ax[0, 1].set_xlabel("Total Profit ($)")
ax[0, 1].set_ylabel("Category")

for i, value in enumerate(grouped_profit_by_category.values):
    ax[0, 1].text(value * 1.01, i, f"{value:,.0f}", va="center")


# Total Costs by Category
grouped_cost_by_category = (
    df_events_products_countries.groupby("category")["total_cost"]
    .sum().sort_values(ascending=True)
)

ax[1, 0].barh(grouped_cost_by_category.index, grouped_cost_by_category.values, edgecolor="black")
ax[1, 0].set_title("Total Costs by Category")
ax[1, 0].set_xlabel("Total Costs ($)")
ax[1, 0].set_ylabel("Category")

for i, value in enumerate(grouped_cost_by_category.values):
    ax[1, 0].text(value * 1.01, i, f"{value:,.0f}", va="center")


# Total Units Sold by Category
grouped_sold_by_category = (
    df_events_products_countries.groupby("category")["units_sold"]
    .sum().sort_values(ascending=True)
)

ax[1, 1].barh(grouped_sold_by_category.index, grouped_sold_by_category.values, edgecolor="black")
ax[1, 1].set_title("Total Units Sold by Category")
ax[1, 1].set_xlabel("Total Units Sold")
ax[1, 1].set_ylabel("Category")

for i, value in enumerate(grouped_sold_by_category.values):
    ax[1, 1].text(value * 1.01, i, f"{value:,.0f}", va="center")


plt.tight_layout()
plt.show()




### **Sales Analysis by Product Categories**

**Most popular categories by units sold:**
Office Supplies, Beverages, Fruits, Clothes, Vegetables, Baby Food.

**Top revenue-generating categories:**
Office Supplies, Household, Cosmetics, Meat, Baby Food.

**Most profitable categories:**
Cosmetics, Office Supplies, Household, Baby Food, Clothes.

**Categories with the highest costs:**
Office Supplies, Household, Meat, Cosmetics, Baby Food.

**Recommendations:**

* **Focus on profitability:** Increase profit margins in popular but less profitable categories (e.g., Beverages).
* **Develop top performers:** Invest in marketing and promotion for the most profitable categories (Cosmetics, Office Supplies, Household).



## **Sales Analysis (revenue, costs, profits, product popularity) by geography (countries, regions).**

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 10))

#Country "Unknown" excluded to avoid distortion
df_geo = df_events_products_countries[
    df_events_products_countries["country"] != "Unknown"

]
# Top 10 countries by REVENUE

grouped_revenue_by_country = (
    df_geo.groupby("country")["total_revenue"]
    .sum().sort_values(ascending=True).tail(10)
)

ax[0, 0].barh(grouped_revenue_by_country.index, grouped_revenue_by_country.values, color="salmon", edgecolor="black")
ax[0, 0].set_title("Top 10 countries by Revenue")
ax[0, 0].set_xlabel("Total Revenue ($)")
ax[0, 0].set_ylabel("Country")

for i, value in enumerate(grouped_revenue_by_country.values):
    ax[0, 0].text(value * 1.01, i, f"{value:,.0f}", va="center")

#  Top 10 countries by Profit
grouped_profit_by_country = (
    df_geo.groupby("country")["total_profit"]
    .sum().sort_values(ascending=True).tail(10)
)

ax[0, 1].barh(grouped_profit_by_country.index, grouped_profit_by_country.values, color="salmon", edgecolor="black")
ax[0, 1].set_title("Top 10 countries by Profit")
ax[0, 1].set_xlabel("Total Profit ($)")
ax[0, 1].set_ylabel("Country")

for i, value in enumerate(grouped_profit_by_country.values):
    ax[0, 1].text(value * 1.01, i, f"{value:,.0f}", va="center")


# Top 10 countries by COSTS
grouped_cost_by_country = (
    df_geo.groupby("country")["total_cost"]
    .sum().sort_values(ascending=True).tail(10)
)

ax[1, 0].barh(grouped_cost_by_country.index, grouped_cost_by_country.values, color="salmon", edgecolor="black")
ax[1, 0].set_title("Top 10 countries by COSTS")
ax[1, 0].set_xlabel("Total Costs ($)")
ax[1, 0].set_ylabel("Country")

for i, value in enumerate(grouped_cost_by_country.values):
    ax[1, 0].text(value * 1.01, i, f"{value:,.0f}", va="center")

# Top 10 countries by UNITS SOLD
grouped_sold_by_country = (
    df_geo.groupby("country")["units_sold"]
    .sum().sort_values(ascending=True).tail(10)
)

ax[1, 1].barh(grouped_sold_by_country.index, grouped_sold_by_country.values, color="salmon", edgecolor="black")
ax[1, 1].set_title("Top 10 countries by UNITS SOLD")
ax[1, 1].set_xlabel("Total Units Sold")
ax[1, 1].set_ylabel("Country")

for i, value in enumerate(grouped_sold_by_country.values):
    ax[1, 1].text(value * 1.01, i, f"{value:,.0f}", va="center")



plt.tight_layout()
plt.show()

In [ ]:
from ipywidgets import interact, widgets
countries = sorted(df_geo['country'].dropna().astype(str).unique())

# Function for plotting a graph for a selected country
def plot_profit_by_country(country):
    data = df_geo[df_geo['country'] == country]
    grouped = data.groupby('order_date')['total_profit'].sum().reset_index()

    plt.figure(figsize=(12, 6))
    plt.plot(grouped['order_date'], grouped['total_profit'], marker='o', color='skyblue')
    plt.title(f'Profit by Country by Order Date: {country}', fontsize=16)
    plt.xlabel('Month')
    plt.ylabel('Profit')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Dropdown
interact(plot_profit_by_country,
    country=widgets.Dropdown(options=countries, description='Country:'));

In [ ]:
# Region "Unknown" excluded to avoid distortion
df_region = df_events_products_countries[df_events_products_countries["region"] != "Unknown"]

region_metrics = df_region.groupby("region").agg({
    "total_revenue": "sum",
    "total_cost": "sum",
    "total_profit": "sum",
    "units_sold": "sum"
})

# Create positions for groups
regions = region_metrics.index.tolist()
x = np.arange(len(regions))  # positions of regions on x-axis
width = 0.2  # weigh of bars

fig, ax = plt.subplots(figsize=(10,6))

# Grouped bars
ax.bar(x - 1.5*width, region_metrics["total_revenue"], width, label="Revenue($)")
ax.bar(x - 0.5*width, region_metrics["total_cost"], width, label="Cost($)")
ax.bar(x + 0.5*width, region_metrics["total_profit"], width, label="Profit($)")
ax.bar(x + 1.5*width, region_metrics["units_sold"], width, label="Units Sold")

# Labels
ax.set_xticks(x)
ax.set_xticklabels(regions)
ax.set_ylabel("Values")
ax.set_title("Revenue, Cost, Profit and Units Sold by Region")
ax.legend()

# Value labels for the bars
for i in range(len(regions)):
    for j, metric in enumerate(["total_revenue", "total_cost", "total_profit", "units_sold"]):
        value = region_metrics.iloc[i][metric]
        ax.text(
            x[i] + (j-1.5)*width, value*1.01, f"{value:,.0f}", ha='center', va='bottom', fontsize=8
        )

plt.show()


In [ ]:
# Subregion "Unknown" excluded to avoid distortion
df_subregion = df_events_products_countries[df_events_products_countries["sub_region"] != "Unknown"]

subregion_metrics = df_subregion.groupby("sub_region").agg({
    "total_revenue": "sum",
    "total_cost": "sum",
    "total_profit": "sum",
    "units_sold": "sum"
}).sort_values(by="total_revenue", ascending=False).tail(5)

# Create positions for groups
sub_regions = subregion_metrics.index.tolist()
x = np.arange(len(sub_regions))  # positions of subregions on x-axis
width = 0.2  # weigh of bars

fig, ax = plt.subplots(figsize=(15,6))

# Grouped bars
ax.bar(x - 1.5*width, subregion_metrics["total_revenue"], width, label="Revenue($)")
ax.bar(x - 0.5*width, subregion_metrics["total_cost"], width, label="Cost($)")
ax.bar(x + 0.5*width, subregion_metrics["total_profit"], width, label="Profit($)")
ax.bar(x + 1.5*width, subregion_metrics["units_sold"], width, label="Units Sold")

# Labels
ax.set_xticks(x)
ax.set_xticklabels(sub_regions)
ax.set_ylabel("Values")
ax.set_title("Revenue, Cost, Profit and Units Sold by Subregion")
ax.legend()

# Value labels for the bars
for i in range(len(sub_regions)):
    for j, metric in enumerate(["total_revenue", "total_cost", "total_profit", "units_sold"]):
        value = subregion_metrics.iloc[i][metric]
        ax.text(
            x[i] + (j-1.5)*width, value*1.01, f"{value:,.0f}", ha='center', va='bottom', fontsize=8
        )

plt.show()



### **Company Activity by Geography**

The most profitable countries are: **Czech Republic, Ukraine, Bosnia and Herzegovina, Portugal, and Macedonia**.

The countries with the highest costs are the same ones that generate the highest revenue: **Czech Republic, Ukraine, Bosnia and Herzegovina, Portugal, and Macedonia**.

The leaders by number of sales are: **Macedonia, San Marino, Andorra, Greece, and Malta**.

The most profitable countries are: **Andorra, Ukraine, Malta, San Marino, and Hungary**.

The leading region is **Europe** (448.5M USD), particularly ***Southern Europe*** subregion (166.2M USD).


**Recommendations:**
Adopt a market expansion strategy and strengthen marketing presence and logistics in **Southern Europe**, leveraging the strong performance in the leading countries as a foundation for further growth.


## **Sales analysis (revenue, costs, profits, product popularity) by sales channel (online or offline).**

In [ ]:
#Normalization of data

df_events_products_countries["sales_channel"].value_counts()


In [ ]:
df_events_products_countries["sales_channel"] = (
    df_events_products_countries["sales_channel"]
    .str.strip()
    .str.capitalize()
)

df_events_products_countries["sales_channel"].value_counts()


In [ ]:
# 1. Pivot table for sales channels
channel_pivot = df_events_products_countries.pivot_table(
    index="sales_channel",
    values=["total_revenue", "total_cost", "total_profit"],
    aggfunc="sum"
)

# 2. Positions for groups
channels = channel_pivot.index.tolist()
x = np.arange(len(channels))
width = 0.2

fig, ax = plt.subplots(figsize=(10,6))

# 3. grouped bars
metrics = ["total_revenue", "total_cost", "total_profit"]
colors = ["skyblue", "lightgreen", "salmon"]

for i, metric in enumerate(metrics):
    ax.bar(x + (i-1.5)*width, channel_pivot[metric], width, label=metric, color=colors[i])

# 4. Labels
ax.set_xticks(x)
ax.set_xticklabels(channels)
ax.set_ylabel("Values")
ax.set_title("Revenue($), Cost($), Profit($) and Units Sold by Sale Channels")
ax.legend()

# 5. Bar labels
for i in range(len(channels)):
    for j, metric in enumerate(metrics):
        value = channel_pivot[metric].iloc[i]
        ax.text(
            x[i] + (j-1.5)*width, value*1.01, f"{value:,.0f}", ha='center', va='bottom', fontsize=8
        )

plt.show()

### **Sales Analysis (Online and Offline)**
**Even Diversification**: The number of online and offline orders is almost the same.

**Profitability:** Offline sales bring more profit.

## **Analysis of the time interval between order and shipment by product category, countries, and regions.**

In [ ]:
df_events_products_countries.info()

In [ ]:
df_events_products_countries.head()

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(15, 5))

#Convert datatype
df_events_products_countries["order_date"] = pd.to_datetime(df_events_products_countries["order_date"])
df_events_products_countries["ship_date"] = pd.to_datetime(df_events_products_countries["ship_date"])

#Count difference between ship_date і order_date in days
df_events_products_countries["interval"] = (
    df_events_products_countries["ship_date"] - df_events_products_countries["order_date"]
).dt.days

# Sort categories by Median Shipping Interval
grouped_interval_by_category = (
    df_events_products_countries.groupby("category")["interval"]
    .median()
    .sort_values(ascending=False)
)
category_order = grouped_interval_by_category.index

# Median Shipping Interval by Category
sns.barplot(
    y="category",
    x="interval",
    order=category_order,
    data=df_events_products_countries,
    estimator=np.median,
    orient="h",
    ax=ax[0]
)

# Add labels
for p in ax[0].patches:
    width = p.get_width()
    if width > 0:
        ax[0].text(
            width,
            p.get_y() + p.get_height() / 2,
            f"{width:.0f}",
            ha='left',
            va='bottom',
            fontsize=9,
            color="black"
        )

ax[0].set_title("Median Shipping Interval by Category")
ax[0].set_xlabel("Median interval (days)")
ax[0].set_ylabel("Category")

#----------------------------------------------------------------------
# Sort countries by Median Shipping Interval
grouped_interval_by_country = (
    df_events_products_countries.groupby("country")["interval"]
    .median()
    .sort_values(ascending=False).tail(12)
)
country_order = grouped_interval_by_country.index

# Barchar in the subplot
sns.barplot(
    y="country",
    x="interval",
    order=country_order,
    data=df_events_products_countries,
    estimator=np.median,
    orient="h",
    ax=ax[1]
)

# Add labels
for p in ax[1].patches:
    width = p.get_width()
    if width > 0:
        ax[1].text(
            width,
            p.get_y() + p.get_height() / 2,
            f"{width:.0f}",
            ha='left',
            va='bottom',
            fontsize=9,
            color="black"
        )

ax[1].set_title("Median Shipping Interval by Country")
ax[1].set_xlabel("Median interval (days)")
ax[1].set_ylabel("Country")

#----------------------------------------------------------------------
# Sort regions by Median Shipping Interval
grouped_interval_by_region = (
    df_events_products_countries.groupby("region")["interval"]
    .median()
    .sort_values(ascending=False)
)
region_order = grouped_interval_by_region.index

# Barchar in the third subplot
sns.barplot(
    y="region",
    x="interval",
    order=region_order,
    data=df_events_products_countries,
    estimator=np.median,
    orient="h",
    ax=ax[2]
)

# Add labels
for p in ax[2].patches:
    width = p.get_width()
    if width > 0:
        ax[2].text(
            width,
            p.get_y() + p.get_height() / 2,
            f"{width:.0f}",
            ha='left',
            va='bottom',
            fontsize=9,
            color="black"
        )

ax[2].set_title("Median Shipping Interval by Region")
ax[2].set_xlabel("Median interval (days)")
ax[2].set_ylabel("Region")

plt.tight_layout()
plt.show()



### **Analysis of the Time Interval Between Order Placement and Shipment**

The product categories exhibiting the longest intervals, defined as the number of days between the order date and the shipment date (**Shipping Interval**), are **Cereal, Baby Food, Cosmetics, Office Supplies, and Snacks**. *These categories may require more complex processing, packaging, or logistical coordination, contributing to extended fulfillment times.*

The countries with the highest Shipping Interval are: **Ireland, Latvia, Denmark, Iceland, and San Marino**.

Across regions, the Shipping Interval is relatively consistent, ranging **from 24 to 26 days.**

## **Analysis of the dependence of profit on the time required to ship goods.**

In [ ]:
#PROFIT
grouped_profit_by_category = (
    df_events_products_countries.groupby("category")["total_profit"]
    .sum()
)

# INTERVAL
grouped_interval_by_category = (
    df_events_products_countries.groupby("category")["interval"]
    .median()
)

df_scatter = pd.DataFrame({
    "profit": grouped_profit_by_category,
    "interval": grouped_interval_by_category
})


# --- Scatter chart ---
plt.figure(figsize=(10,5))
plt.scatter(df_scatter["interval"], df_scatter["profit"], color="blue", alpha=0.5)

plt.xlabel("Median Interval (days)")
plt.ylabel("Total profit ($)")
plt.title("Dependency: Profit vs Shipping Interval")

# --- Money formatting on Y axis ---
def money(x, pos):
    return f'${x:,.0f}'

plt.gca().yaxis.set_major_formatter(FuncFormatter(money))


# Labels for the categories
for category, row in df_scatter.iterrows():
    plt.text(row["interval"], row["profit"], category, fontsize=8)

plt.tight_layout()
plt.show()


### **Analysis of the Relationship Between the Order–Shipment Interval and Profit**

The analysis indicates **no direct correlation** between the time interval from order placement to shipment and the resulting profit. Profitability does not appear to be influenced by how long it takes to process and ship an order.

Instead, the primary drivers of profit are factors such as **customer demand, product quality, and brand recognition**. These elements play a significantly larger role in determining sales volume and profitability.

**Recommendations:**
Focus on strengthening brand presence by investing in advertising and marketing activities. Enhancing brand visibility and perceived value may increase customer willingness to wait for products, even when shipping intervals are longer.



## **Analysis of sales dynamics (over time) by product categories, countries, regions, identification of main trends.**

In [ ]:
revenue_by_year = (df_events_products_countries.groupby(df_events_products_countries["order_date"].dt.to_period("Y"))["total_revenue"].sum())
revenue_by_year.index = revenue_by_year.index.to_timestamp()

plt.figure(figsize=(10, 6))
plt.plot(revenue_by_year.index, revenue_by_year.values, marker='o', linestyle='-')
plt.title("Total Revenue by Year")
plt.xlabel('Year')
plt.ylabel('Total Revenue ($)')

# --- Money formatting on Y axis ---
def money(x, pos):
    return f'${x:,.0f}'

plt.gca().yaxis.set_major_formatter(FuncFormatter(money))
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
# Add column to dataframe "year"
df_events_products_countries["year"] = df_events_products_countries["order_date"].dt.year

# Filter years from 2014 to 2017
df_years = df_events_products_countries[df_events_products_countries["year"].between(2014, 2017)]

# Group revenue by categories and years
revenue_by_category_year = (
    df_years.groupby(["category", "year"])["total_revenue"]
    .sum()
    .reset_index()
)

# Find top-5 categories by total revenue from 2014 to 2017
top5_categories = (
    revenue_by_category_year.groupby("category")["total_revenue"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

# Pivot table
pivot_df_categories = revenue_by_category_year.pivot(index="category", columns="year", values="total_revenue")

# Filter top 5 categories
pivot_df_categories = pivot_df_categories.loc[top5_categories]

# Buiding the chart
pivot_df_categories.plot(
    kind="bar",
    figsize=(12,6),
    edgecolor="black"
)

plt.title("Top 5 Categories by Revenue (2014–2017)")
plt.xlabel("Category")
plt.ylabel("Revenue ($)")
plt.xticks(rotation=0, ha="right")
plt.legend(title="Year")

# --- Money formatting on Y axis ---
def money(x, pos):
    return f'${x:,.0f}'

plt.gca().yaxis.set_major_formatter(FuncFormatter(money))

plt.tight_layout()
plt.show()



In [ ]:
print(pivot_df_categories)

In [ ]:
# Add column "year" to dataframe without "Unknown" country
df_geo = df_geo.copy()
df_geo["year"] = df_geo["order_date"].dt.year

# Filter years from 2014 to 2017
df_years_countries = df_geo[df_geo["year"].between(2014, 2017)]

# Group revenue by countries and years
revenue_country_year = (
    df_years_countries.groupby(["country", "year"])["total_revenue"]
    .sum()
    .reset_index()
)

# Find the top 10 countries by total revenue from 2014 to 2017
top10_countries = (
    revenue_country_year.groupby("country")["total_revenue"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
    .index
)

# Pivot table
pivot_df_countries = revenue_country_year.pivot(index="country", columns="year", values="total_revenue")

# Filter the top 10 countries
pivot_df_countries = pivot_df_countries.loc[top10_countries]

# Bar chart
pivot_df_countries.plot(
    kind="bar",
    figsize=(15,6),
    edgecolor="black"
)

plt.title("Top 10 Countries by Revenue (2014–2017)")
plt.xlabel("Country")
plt.ylabel("Revenue ($)")
plt.xticks(rotation=45, ha="right")
plt.legend(title="Year")

# --- Money formatting on Y axis ---
def money(x, pos):
    return f'${x:,.0f}'

plt.gca().yaxis.set_major_formatter(FuncFormatter(money))


plt.tight_layout()
plt.show()

In [ ]:
print(pivot_df_countries)

In [ ]:
#  Add column "year" for dataframe withot "Unknown" region
df_region = df_region.copy()
df_region["year"] = df_region["order_date"].dt.year

# Filter years 2014 - 2017
df_years_region = df_region[df_region["year"].between(2014, 2017)]


# Group revenue by regions and years
revenue_regions_year = (
    df_years_region.groupby(["region", "year"])["total_revenue"]
    .sum()
    .reset_index()
)

# Pivot table
pivot_df = revenue_regions_year.pivot(index="region", columns="year", values="total_revenue")


# Bar chart
pivot_df.plot(
    kind="bar",
    figsize=(12,6),
    edgecolor="black"
)

plt.title("Revenue by Regions (2014–2017)")
plt.xlabel("Region")
plt.ylabel("Revenue ($)")
plt.xticks(rotation=0, ha="right")
plt.legend(title="Year")

# --- Money formatting on Y axis ---
def money(x, pos):
    return f'${x:,.0f}'

plt.gca().yaxis.set_major_formatter(FuncFormatter(money))

plt.tight_layout()
plt.show()

In [ ]:
print(pivot_df)


**Analysis of Sales Dynamics (Over Time) by Product Categories, Countries, and Regions, and Identification of Key Trends**

Based on the analysis of the period from **2012 to 2017**, we observe an overall decline in sales. The peak occurred in **2012**, followed by a significant increase in **2014**, and then a sharp downturn continuing through **2017**.

To analyze sales dynamics by product categories, countries, and regions, a data subset for **2014–2017** was extracted.

Over the last four years, a general downward trend in sales is visible. The only exceptions are the **Cosmetics** and **Snacks** categories, which show growth.

There are no consistently leading countries — the top purchasers change from year to year. Although most countries demonstrate declining sales, there is an upward movement in **2017** for several countries, including *Macedonia, Slovenia,* and *Latvia*.

The analysis of revenue dynamics across regions also reveals a declining trend.





## **Analysis of sales by day of the week. Identification of seasonality of products.**

In [ ]:
weekday_order = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

df_events_products_countries["day"] = pd.Categorical(
    df_events_products_countries["order_date"].dt.day_name(),
    categories=weekday_order,
    ordered=True
)

# Filter years from 2014 to 2017
df_years = df_events_products_countries[df_events_products_countries["year"].between(2014, 2017)]

# Pivot table
pivot_table_weekday = pd.pivot_table(df_years, values="units_sold", index="day", aggfunc="sum", observed=False)

# Buiding the bar chart
pivot_table_weekday.plot(
    kind="bar",
    figsize=(12,6),
    edgecolor="black"
)

plt.title("Units Sold by Weekdays (2014-2017)")
plt.xlabel("Weekday")
plt.ylabel("Units sold")
plt.xticks(rotation=0, ha="right")

plt.tight_layout()
plt.show()


In [ ]:
print(pivot_table_weekday)

In [ ]:

# Find top-5 categories by order_id
top5_categories_season = (
    df_years.groupby("category")["units_sold"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

# Filter dataframe for only top-5 categories
df_top5_categories_season = df_years[df_years["category"].isin(top5_categories_season)]


# Group order_id by day and category
pivot_table = pd.pivot_table(
    df_top5_categories_season,
    values="units_sold",
    index="day",
    columns="category",
    aggfunc="sum",
    observed=False
)

# Build the bar chart
pivot_table.plot(
    kind="bar",
    figsize=(15, 6),
    edgecolor="black"
)

plt.title("Units Sold by Weekdays by Top 5 categories 2014-2017")
plt.xlabel("Weekday")
plt.ylabel("Units Sold")
plt.xticks(rotation=0, ha="right")
plt.legend(title="Category")


plt.tight_layout()
plt.show()

In [ ]:
print(pivot_table)

### **Sales Analysis by Day of the Week**

**Overall Sales Overview**

**Highest Sales:**

* **Monday** (near 502,000 units) and **Sunday** (~495,000 units) are the peak days.
* This indicates that customers tend to shop actively at the start of the week and on weekends.

**Lowest Sales:**

* **Friday** (~372,000 units) — the lowest performance.
* This may indicate that people are busy preparing for the weekend.

**Mid-range Sales:**

* Tuesday, Wednesday, Thursday, and Saturday show moderate sales in the range of 395,000–435,000 units.

---

 **Top 5 Categories by Day of the Week**

 **Beverages**

* Stable demand throughout the week (35,000-60,000 units).
* Peak on Monday (~60,000).
* Lowest on Wednesday (~35,000).

 **Clothes**

* Clear intra-week seasonality.
* **Tuesday** - unusually high demand (~70,000 units).
* **Thursday** (near 52,000) and **Saturday** (~49,000) also show strong sales.
* Lowest sales midweek and on Friday.

 **Fruits**

* **Sunday** is the absolute leader (~77,000 units).
* Monday and Saturday also show high demand (62,000-63,000).
* Sharp drop on Tuesday (near 16,000) and Friday (~11,000).
* This indicates that people prefer buying fresh fruits for the weekend.

 **Office Supplies**

* **Sunday** (near 69,000) and **Monday** (~65,000) show the highest sales.
* Indicates preparation for the workweek.
* **Friday** also shows an increase (~56,000) - stocking up before the weekend.
* Wednesday (~46,000) is the lowest point.

 **Vegetables**

* **Sunday** (near 57,000) and **Wednesday** (~55,000) are peak days.
* The lowest sales are on Thursday (~7,000) — an anomalous decline.
* This may suggest that customers buy fresh vegetables twice a week.

---

## **Key Insights**

1. **Monday and Sunday** are critical days for retail performance.
2. **Friday** is the weakest day — promotional activities should be increased.
3. **Different categories peak on different days:**

   * Clothes sell best on Tuesday.
   * Fresh products (fruits, vegetables) peak on weekends.
   * Office supplies — Sunday and Monday.
4. **Recommendations:**

   * Optimize staffing and stock levels for Monday and Sunday.
   * Enhance marketing strategies on Friday to boost sales.



In [ ]:
month_order = ["January", "February", "March", "April", "May", "June",
               "July", "August", "September", "October", "November", "December"]

# Add column to dataframe 2014 - 2017 years "month"
df_years = df_years.copy()
df_years["month"] = pd.Categorical(
    df_years["order_date"].dt.month_name(),
    categories=month_order,
    ordered=True
)

# Pivot for heatmap: categories × months
pivot_heatmap = pd.pivot_table(
    df_years,
    values="units_sold",
    index="category",
    columns="month",
    aggfunc="sum",
    observed=False
)




plt.figure(figsize=(14,7))
sns.heatmap(pivot_heatmap, annot=False, cmap="Blues")

plt.title("Seasonality Heatmap for Categories by Units_sold (2014-2017)")
plt.xlabel("Month")
plt.ylabel("Category")



plt.tight_layout()
plt.show()


In [ ]:
print(pivot_heatmap)


### **Seasonality Analysis of Products Based on Units Sold**

The heatmap reveals the following patterns:

 **Categories with Strong Seasonality**

**Baby Food**

* Highest sales in January
* Consistently high demand during spring and summer
* Likely related to childbirth cycles and seasonal needs

**Beverages**

* Sales peak in February (over 60,000 units)
* High demand during the summer period
* Winter holidays may also stimulate sales

**Clothes**

* Distinct peaks in January and June (around 50,000 units)
* Corresponds to seasonal sales and wardrobe changes

**Vegetables**

* High demand in March and from May to August
* Related to harvest seasons and availability of fresh produce

**Fruits**

* Strong sales in January, March, and November
* Seasonal availability affects consumer demand

**Office Supplies**

* Peaks in March, May, and November
* Possibly linked to the start of the academic and financial year

---

 **Categories with Stable Demand**

**Cereal, Household, Personal Care, Snacks, Meat**

* Relatively stable demand throughout the year
* Minor fluctuations indicate their status as essential goods

**Cosmetics**

* Moderate seasonality with increased demand in March and July
* Likely connected to holidays and the summer season

---

Overall, essential goods show stability, while seasonal and holiday-oriented categories display pronounced sales peaks.

---

 **General Conclusions and Recommendations**

* Essential goods demonstrate stable demand, so a standard inventory management strategy is sufficient.
* Seasonal and holiday-driven categories require forecasting and preparation of stock in advance.
* Developing seasonal promotions and marketing campaigns can boost sales during peak periods.
* Annual monitoring of demand trends will support timely adjustments in procurement and sales strategies.


In [ ]:
month_order = ["January", "February", "March", "April", "May", "June",
               "July", "August", "September", "October", "November", "December"]

# Add column to dataframe 2014 - 2017 years "month"
df_years = df_years.copy()
df_years["month"] = pd.Categorical(
    df_years["order_date"].dt.month_name(),
    categories=month_order,
    ordered=True
)

# Pivot for heatmap: categories × months
pivot_heatmap_revenue = pd.pivot_table(
    df_years,
    values="total_revenue",
    index="category",
    columns="month",
    aggfunc="sum",
    observed=False
)


plt.figure(figsize=(15,7))
sns.heatmap(pivot_heatmap_revenue, annot=False, cmap="Blues")

plt.title("Seasonality Heatmap for Categories by Total Revenue (2014-2017)")
plt.xlabel("Month")
plt.ylabel("Category")


plt.tight_layout()
plt.show()


In [ ]:
print(pivot_heatmap_revenue)

### **Seasonality Analysis of Products Based on Total Revenue.**
1. **Office Supplies**

* Clearly driven by business procurement cycles.
* Marketing efforts should focus on Feb–Mar and Apr–May months.
* Increase inventory before high-demand periods.

2. **Household & Cosmetics**
* These categories show multi-peak seasonal patterns.
* Plan promotional campaigns in late winter and mid-summer.

3. **Meat**
* Strong summer demand → prepare for seasonal surge.
* Marketing around summer barbecue themes is effective.

4. **Stable-demand categories (Cereal, Fruits, Vegetables, etc.)**

* These categories rely more on price competitiveness and distribution reliability, not seasonality.
* Focus on continuous supply chain efficiency.

---

### **1. Most Profitable Categories**

 **Office Supplies — the main revenue driver**

* Peaks in **March, May, September, November, December**
* High revenue due to **B2B corporate orders and high-ticket items**
  →  *The most critical category for the business.*

 **Household — strong seasonal revenue**

* Peaks in **June–August** (summer repairs and home upgrades)
  →  *A major revenue source during summer months.*

 **Cosmetics — stable and premium**

* Peaks in **March, June–July, November**
  →  *High potential for growth through premium lines.*

---

### **2. High-volume but low-revenue categories**

 **Beverages, Vegetables, Fruits, Personal Care**

* High order volume but low average price → low revenue.
  →  *Consume operational resources without generating significant profit.*

**Snacks, Cereal**

* Minimal contribution to revenue.
  →  *Kept mainly for assortment breadth.*

---

### **3. Key Strategic Contrasts**

| Category                        | Volume | Revenue   | Conclusion               |
| ------------------------------- | ------ | --------- | ------------------------ |
| Office Supplies                 | Medium | ⭐ Highest | B2B, high-ticket items   |
| Household                       | Medium | ⭐ High    | Seasonal large purchases |
| Cosmetics                       | Medium | 🔥 High   | Premium products         |
| Beverages / Fruits / Vegetables | High   | ⚠️ Low    | Low-margin essentials    |
| Baby Food                       | High   | Medium    | Stable category          |

---

### **4. Strategic Insights**

**A. Focus on the categories that generate real money**

* Office Supplies
* Household
* Cosmetics
   
   → *These are the revenue core. Invest and scale.*

---

**B. Re-evaluate low-revenue categories**

* Beverages, Vegetables, Fruits, Personal Care
   
   → *Increase prices by 10-15% or introduce a minimum order amount.*

---

###**C. Seasonal revenue strategy**

* **March** — strongest month of the year (Office Supplies, Cosmetics)
* **Summer (June-August)** - Household
* **November-December** - Office Supplies, Household, Cosmetics
  
   → *Plan marketing and inventory around these peaks.*

---

### **D. Strengthen the B2B segment**

Office Supplies shows that corporate clients drive
the highest revenue.

 → *Introduce special conditions, account managers, automated reorders.*

---

### **Conclusion**

**The business relies heavily on high-volume, low-margin categories, but most profit comes from Office Supplies, Household, and Cosmetics.**

 **Strategy: reduce emphasis on low-revenue categories and increase investment in premium and B2B-focused segments.**



